In [4]:
#initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
import qiskit
from qiskit import IBMQ, Aer, assemble, transpile
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import *

from qiskit.providers.ibmq import least_busy
IBMQ.save_account('9ff1c619f266188a92c5a0444fed8fd525e7889a3e4aee6f0d75e7d94c71ba57dae5e0a3258a28365e8645e1e4bf9c8983d34cfec777358a2c7fa08ee5b6a48b', overwrite=True)

# import basic plot tools
from qiskit.visualization import plot_histogram

# Run our circuit on the least busy backend. Monitor the execution of the job in the queue
from qiskit.tools.monitor import job_monitor

from qiskit.providers.aer import AerSimulator
import random


In [5]:
conditions = [[0,1,2,3,4,5],[6,7,8,9,10,11], [12,13,14,15,16,17],[0,1,6,7,12,13],[2,3,8,9,14,15],[4,5,10,11,16,17]]

In [6]:
def condition_1(qc, a, b, c, d, e, f, output):
    qc.ccx(a, d, output)
    qc.ccx(a, f, output)
    qc.ccx(b, c, output)
    qc.ccx(b, e, output)
    qc.ccx(c, f, output)
    qc.ccx(d, e, output)

In [7]:
def condition_2(qc, a, b, c, d, e, f, output):
    qc.ccx(a, b, output)
    qc.ccx(c, d, output)
    qc.ccx(e, f, output)
    qc.x(output)

In [8]:
#trial:
in_qubits = QuantumRegister(6, name = 'inputs')
out_qubit = QuantumRegister(3, name = 'outputs')
qc = QuantumCircuit(in_qubits, out_qubit)
condition_1(qc, in_qubits[0], in_qubits[1],  in_qubits[2],  in_qubits[3],  in_qubits[4],  in_qubits[5], out_qubit[0])
condition_2(qc, in_qubits[0], in_qubits[1],  in_qubits[2],  in_qubits[3],  in_qubits[4],  in_qubits[5], out_qubit[1])
qc.ccx(out_qubit[0], out_qubit[1], out_qubit[2])

qc.draw()

inputs_0: ──■────■───────────────────■───────────────────────────
             │    │                   │                           
 inputs_1: ──┼────┼────■────■─────────■───────────────────────────
             │    │    │    │         │                           
 inputs_2: ──┼────┼────■────┼────■────┼─────────■─────────────────
             │    │    │    │    │    │         │                 
 inputs_3: ──■────┼────┼────┼────┼────┼────■────■─────────────────
             │    │    │    │    │    │    │    │                 
 inputs_4: ──┼────┼────┼────■────┼────┼────■────┼────■────────────
             │    │    │    │    │    │    │    │    │            
 inputs_5: ──┼────■────┼────┼────■────┼────┼────┼────■────────────
           ┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐┌─┴─┐  │  ┌─┴─┐  │    │            
outputs_0: ┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├──┼──┤ X ├──┼────┼─────────■──
           └───┘└───┘└───┘└───┘└───┘┌─┴─┐└───┘┌─┴─┐┌─┴─┐┌───┐  │  
outputs_1: ─────────────────────────┤ X ├─────┤ X ├┤ X ├┤ X ├──■──
                                    └───┘     └───┘└───┘└───┘┌─┴─┐
outputs_2: ──────────────────────────────────────────────────┤ X ├
                                                             └───┘

In [9]:
in_qubits = QuantumRegister(18, name='inputs')
condition_qubits = QuantumRegister(12, name ='c')
out_qubit = QuantumRegister(1, name = 'output')
qc = QuantumCircuit(in_qubits, condition_qubits, out_qubit)

i = 0
for condition in conditions:
    condition_1(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i])
    condition_2(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i+1])
    i += 2

qc.mct(condition_qubits, out_qubit)
qc.draw()

»
 inputs_0: ──■──────────────■───────────────────────────────────────────────»
             │              │                                               »
 inputs_1: ──┼──────────────┼──────────────■──────────────■─────────────────»
             │              │              │              │                 »
 inputs_2: ──┼──────────────┼──────────────■──────────────┼──────────────■──»
             │              │              │              │              │  »
 inputs_3: ──■──────────────┼──────────────┼──────────────┼──────────────┼──»
             │              │              │              │              │  »
 inputs_4: ──┼──────────────┼──────────────┼──────────────■──────────────┼──»
             │              │              │              │              │  »
 inputs_5: ──┼──────────────■──────────────┼──────────────┼──────────────■──»
             │              │              │              │              │  »
 inputs_6: ──┼────■─────────┼────■─────────┼──────────────┼──────────────┼──»
             │    │         │    │         │              │              │  »
 inputs_7: ──┼────┼─────────┼────┼─────────┼────■─────────┼────■─────────┼──»
             │    │         │    │         │    │         │    │         │  »
 inputs_8: ──┼────┼─────────┼────┼─────────┼────■─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
 inputs_9: ──┼────■─────────┼────┼─────────┼────┼─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_10: ──┼────┼─────────┼────┼─────────┼────┼─────────┼────■─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_11: ──┼────┼─────────┼────■─────────┼────┼─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_12: ──┼────┼────■────┼────┼────■────┼────┼─────────┼────┼─────────┼──»
             │    │    │    │    │    │    │    │         │    │         │  »
inputs_13: ──┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼────■────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_14: ──┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_15: ──┼────┼────■────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_16: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_17: ──┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼────┼──»
           ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐»
      c_0: ┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├»
           └───┘  │    │  └───┘  │    │  └───┘  │    │  └───┘  │    │  └───┘»
      c_1: ───────┼────┼─────────┼────┼─────────┼────┼─────────┼────┼───────»
                ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       »
      c_2: ─────┤ X ├──┼───────┤ X ├──┼───────┤ X ├──┼───────┤ X ├──┼───────»
                └───┘  │       └───┘  │       └───┘  │       └───┘  │       »
      c_3: ────────────┼──────────────┼──────────────┼──────────────┼───────»
                     ┌─┴─┐          ┌─┴─┐          ┌─┴─┐          ┌─┴─┐     »
      c_4: ──────────┤ X ├──────────┤ X ├──────────┤ X ├──────────┤ X ├─────»
                     └───┘          └───┘          └───┘          └───┘     »
      c_5: ─────────────────────────────────────────────────────────────────»
                                                                            »
      c_6: ─────────────────────────────────────────────────────────────────»
                                                                            »
      c_7: ─────────────────────────────────────────────────────────────────»
                    

In [10]:
#uncomputation
in_qubits = QuantumRegister(18, name='inputs')
condition_qubits = QuantumRegister(12, name ='c')
out_qubit = QuantumRegister(1, name = 'final')
qc = QuantumCircuit(in_qubits, condition_qubits, out_qubit)

def sukoku_oracle(qc,conditions, condition_qubits, out_qubits):
    i = 0
    for condition in conditions:
        condition_1(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i])
        condition_2(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i+1])
        i += 2

    qc.mct(condition_qubits, out_qubit)
    i=0
    for condition in conditions:
        condition_2(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i+1])
        condition_1(qc, condition[0], condition[1],  condition[2], condition[3], condition[4], condition[5], condition_qubits[i])
        i += 2

sukoku_oracle(qc,conditions, condition_qubits, out_qubit)
qc.draw()

»
 inputs_0: ──■──────────────■───────────────────────────────────────────────»
             │              │                                               »
 inputs_1: ──┼──────────────┼──────────────■──────────────■─────────────────»
             │              │              │              │                 »
 inputs_2: ──┼──────────────┼──────────────■──────────────┼──────────────■──»
             │              │              │              │              │  »
 inputs_3: ──■──────────────┼──────────────┼──────────────┼──────────────┼──»
             │              │              │              │              │  »
 inputs_4: ──┼──────────────┼──────────────┼──────────────■──────────────┼──»
             │              │              │              │              │  »
 inputs_5: ──┼──────────────■──────────────┼──────────────┼──────────────■──»
             │              │              │              │              │  »
 inputs_6: ──┼────■─────────┼────■─────────┼──────────────┼──────────────┼──»
             │    │         │    │         │              │              │  »
 inputs_7: ──┼────┼─────────┼────┼─────────┼────■─────────┼────■─────────┼──»
             │    │         │    │         │    │         │    │         │  »
 inputs_8: ──┼────┼─────────┼────┼─────────┼────■─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
 inputs_9: ──┼────■─────────┼────┼─────────┼────┼─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_10: ──┼────┼─────────┼────┼─────────┼────┼─────────┼────■─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_11: ──┼────┼─────────┼────■─────────┼────┼─────────┼────┼─────────┼──»
             │    │         │    │         │    │         │    │         │  »
inputs_12: ──┼────┼────■────┼────┼────■────┼────┼─────────┼────┼─────────┼──»
             │    │    │    │    │    │    │    │         │    │         │  »
inputs_13: ──┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼────■────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_14: ──┼────┼────┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_15: ──┼────┼────■────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_16: ──┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────┼────■────┼──»
             │    │    │    │    │    │    │    │    │    │    │    │    │  »
inputs_17: ──┼────┼────┼────┼────┼────■────┼────┼────┼────┼────┼────┼────┼──»
           ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐  │    │  ┌─┴─┐»
      c_0: ┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├──┼────┼──┤ X ├»
           └───┘  │    │  └───┘  │    │  └───┘  │    │  └───┘  │    │  └───┘»
      c_1: ───────┼────┼─────────┼────┼─────────┼────┼─────────┼────┼───────»
                ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       »
      c_2: ─────┤ X ├──┼───────┤ X ├──┼───────┤ X ├──┼───────┤ X ├──┼───────»
                └───┘  │       └───┘  │       └───┘  │       └───┘  │       »
      c_3: ────────────┼──────────────┼──────────────┼──────────────┼───────»
                     ┌─┴─┐          ┌─┴─┐          ┌─┴─┐          ┌─┴─┐     »
      c_4: ──────────┤ X ├──────────┤ X ├──────────┤ X ├──────────┤ X ├─────»
                     └───┘          └───┘          └───┘          └───┘     »
      c_5: ─────────────────────────────────────────────────────────────────»
                                                                            »
      c_6: ─────────────────────────────────────────────────────────────────»
                                                                            »
      c_7: ─────────────────────────────────────────────────────────────────»
                    

In [11]:
def diffuser(nqubits):
    qc = QuantumCircuit(nqubits)
    # Apply transformation |s> -> |00..0> (H-gates)
    for qubit in range(nqubits):
        qc.h(qubit)
    # Apply transformation |00..0> -> |11..1> (X-gates)
    for qubit in range(nqubits):
        qc.x(qubit)
    # Do multi-controlled-Z gate
    qc.h(nqubits-1)
    qc.mct(list(range(nqubits-1)), nqubits-1)  # multi-controlled-toffoli
    qc.h(nqubits-1)
    # Apply transformation |11..1> -> |00..0>
    for qubit in range(nqubits):
        qc.x(qubit)
    # Apply transformation |00..0> -> |s>
    for qubit in range(nqubits):
        qc.h(qubit)
    # We will return the diffuser as a gate
    U_s = qc.to_gate()
    U_s.name = "U$_s$"
    return U_s

In [12]:
#full algorithm
in_qubits = QuantumRegister(18, name='inputs')
condition_qubits = QuantumRegister(12, name ='c')
out_qubit = QuantumRegister(1, name = 'final')
cbits = ClassicalRegister(18, name='cbits')
qc = QuantumCircuit(in_qubits, condition_qubits, out_qubit, cbits)

qc.initialize([1, -1]/np.sqrt(2), out_qubit)
# Initialize qubits in state |s>
qc.h(in_qubits)
qc.barrier()  # for visual separation
#first iteration
sukoku_oracle(qc,conditions, condition_qubits, out_qubit)
qc.barrier()
qc.append(diffuser(18), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
#second iteration
sukoku_oracle(qc,conditions, condition_qubits, out_qubit)
qc.barrier()
qc.append(diffuser(18), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
#third iteration
sukoku_oracle(qc,conditions, condition_qubits, out_qubit)
qc.barrier()
qc.append(diffuser(18), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
#forth iteration
sukoku_oracle(qc,conditions, condition_qubits, out_qubit)
qc.barrier()
qc.append(diffuser(18), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

qc.draw()




┌───┐               ░                               »
 inputs_0: ─────────────┤ H ├───────────────░───■──────────────■────────────»
                        ├───┤               ░   │              │            »
 inputs_1: ─────────────┤ H ├───────────────░───┼──────────────┼────────────»
                        ├───┤               ░   │              │            »
 inputs_2: ─────────────┤ H ├───────────────░───┼──────────────┼────────────»
                        ├───┤               ░   │              │            »
 inputs_3: ─────────────┤ H ├───────────────░───■──────────────┼────────────»
                        ├───┤               ░   │              │            »
 inputs_4: ─────────────┤ H ├───────────────░───┼──────────────┼────────────»
                        ├───┤               ░   │              │            »
 inputs_5: ─────────────┤ H ├───────────────░───┼──────────────■────────────»
                        ├───┤               ░   │              │            »
 inputs_6: ─────────────┤ H ├───────────────░───┼────■─────────┼────■───────»
                        ├───┤               ░   │    │         │    │       »
 inputs_7: ─────────────┤ H ├───────────────░───┼────┼─────────┼────┼───────»
                        ├───┤               ░   │    │         │    │       »
 inputs_8: ─────────────┤ H ├───────────────░───┼────┼─────────┼────┼───────»
                        ├───┤               ░   │    │         │    │       »
 inputs_9: ─────────────┤ H ├───────────────░───┼────■─────────┼────┼───────»
                        ├───┤               ░   │    │         │    │       »
inputs_10: ─────────────┤ H ├───────────────░───┼────┼─────────┼────┼───────»
                        ├───┤               ░   │    │         │    │       »
inputs_11: ─────────────┤ H ├───────────────░───┼────┼─────────┼────■───────»
                        ├───┤               ░   │    │         │    │       »
inputs_12: ─────────────┤ H ├───────────────░───┼────┼────■────┼────┼────■──»
                        ├───┤               ░   │    │    │    │    │    │  »
inputs_13: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────┼──»
                        ├───┤               ░   │    │    │    │    │    │  »
inputs_14: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────┼──»
                        ├───┤               ░   │    │    │    │    │    │  »
inputs_15: ─────────────┤ H ├───────────────░───┼────┼────■────┼────┼────┼──»
                        ├───┤               ░   │    │    │    │    │    │  »
inputs_16: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────┼──»
                        ├───┤               ░   │    │    │    │    │    │  »
inputs_17: ─────────────┤ H ├───────────────░───┼────┼────┼────┼────┼────■──»
                        └───┘               ░ ┌─┴─┐  │    │  ┌─┴─┐  │    │  »
      c_0: ─────────────────────────────────░─┤ X ├──┼────┼──┤ X ├──┼────┼──»
                                            ░ └───┘  │    │  └───┘  │    │  »
      c_1: ─────────────────────────────────░────────┼────┼─────────┼────┼──»
                                            ░      ┌─┴─┐  │       ┌─┴─┐  │  »
      c_2: ─────────────────────────────────░──────┤ X ├──┼───────┤ X ├──┼──»
                                            ░      └───┘  │       └───┘  │  »
      c_3: ─────────────────────────────────░─────────────┼──────────────┼──»
                                            ░           ┌─┴─┐          ┌─┴─┐»
      c_4: ─────────────────────────────────░───────────┤ X ├──────────┤ X ├»
                                            ░           └───┘          └───┘»
      c_5: ─────────────────────────────────░───────────────────────────────»
                                            ░                               »
      c_6: ─────────────────────────────────░───────────────────────────────»
                                            ░                               »
      c_7: ─────────────────────────────────░─

In [ ]:
# Create extended stabilizer method simulator
extended_stabilizer_simulator = AerSimulator(method='extended_stabilizer')

# Transpile circuit for backend
tqc = transpile(qc, extended_stabilizer_simulator)

extended_stabilizer_result = extended_stabilizer_simulator.run(tqc,
shots=1).result()
counts = extended_stabilizer_result.get_counts()
plot_histogram(counts)
